# Ready, Steady, Go AI (*Tutorial*)

This tutorial is a supplement to the paper, **Ready, Steady, Go AI: A Practical Tutorial on Fundamentals of Artificial Intelligence and Its Applications in Phenomics Image Analysis** (*Patterns, 2021*) by Farid Nakhle and Antoine Harfouche

Read the accompanying paper [here](https://doi.org/10.1016/j.patter.2021.100323).

# Table of contents


* **1. Background**
* **2. Downloading Preprocessed Dataset**
* **3. Importing the Pretrained Densenet-161 DCNN model**
* **4. Generating Explanations with LIME Using Quickshift**
* **5. Generating Explanations with LIME Using Compact-Watershed**

# 1. Background


**How does LIME work?**

The local interpretable model-agnostic explanations (LIME) is an explainable AI algorithm used to open any machine learning (ML) or deep learning (DL) black box model, making their predictions explainable to humans.

Simply put, LIME manipulates input data by deleting parts of it and observes the changes in the output of the black box model to be explained. Through the presence or absence of certain parts of data, it monitors their influence on the classification. This way, the algorithm is capable of generating explanations for various data type including text and imaging data.

To explain imaging data, LIME starts by partitioning an image into multiple segments that share common characteristics such as pixel intensity, known as superpixels. It then generates perturbations by deleting some superpixels in the image and monitors how they affect the prediction of the blackbox model. Finally, it identifies which areas of the image have been important for classification and thus, generates the explanation by highlighting them.

This shows how crucial is the superpixels segmentation for LIME to generate explanations. By default, LIME uses the quickshift algorithm to segment images. 

We will use LIME to explain our three trained models, RF, the standard DCNN, and the pretrained DCNN, using quickshift and Compact-Watershed superpixel segmentation algorithms.


# 2. Downloading Preprocessed Dataset

As a reminder, we are working with the PlantVillage dataset, originally obtained from [here](http://dx.doi.org/10.17632/tywbtsjrjv.1).
For this tutorial, we will be working with a subset of PlantVillage, where we will choose the tomato classes only. We have made the subset available [here](http://dx.doi.org/10.17632/4g7k9wptyd.1). 

The next code will automatically download the preprocessed dataset.

**It is important to note that Colab deletes all unsaved data once the instance is recycled. Therefore, remember to download your results once you run the code.**

In [ ]:
import requests
import os
import zipfile

## FEEL FREE TO CHANGE THESE PARAMETERS
dataset_url = "http://faridnakhle.com/pv/tomato-split-cropped-segmented-balanced.zip"
save_data_to = "/content/dataset/tomato-dataset/"
dataset_file_name = "tomato-split-cropped-segmented-balanced.zip"
#######################################

if not os.path.exists(save_data_to):
    os.makedirs(save_data_to)

r = requests.get(dataset_url, stream = True, headers={"User-Agent": "Ready, Steady, Go AI"})

print("Downloading dataset...")  

with open(save_data_to + dataset_file_name, "wb") as file: 
    for block in r.iter_content(chunk_size = 1024):
         if block: 
             file.write(block)

## Extract downloaded zip dataset file
print("Dataset downloaded")  
print("Extracting files...")  
with zipfile.ZipFile(save_data_to + dataset_file_name, 'r') as zip_dataset:
    zip_dataset.extractall(save_data_to)

## Delete the zip file as we no longer need it
os.remove(save_data_to + dataset_file_name)
print("All done!")  


# 3. Importing the Pretrained Densenet-161 DCNN model

In [ ]:
import argparse
import os
import time

import matplotlib.pyplot as plt

import torch
import numpy as np
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from PIL import Image
from collections import OrderedDict
import json
!pip install lime
from lime import lime_image


In [ ]:
## YOU CAN CHANGE THESE VARIABLES    
EPOCHS = 100
BATCH_SIZE = 20
LEARNING_RATE = 0.0001
data_dir = '/content/dataset/tomato-dataset/'
save_checkpoints = True
save_model_to = '/content/output/'
!mkdir /content/output/
IMG_SIZE = 220
NUM_WORKERS = 1
using_gpu = torch.cuda.is_available()
print_every = 300
ARCH = 'densenet161'
######################################################

Next, we will define a function that creates a data loader for all of our sets (i.e., training, validation, and testing)

In [ ]:
def data_loader(root, batch_size=256, workers=1, pin_memory=True):
    traindir = os.path.join(root, 'train')
    valdir = os.path.join(root, 'val')
    testdir = os.path.join(root, 'test')
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    train_dataset = datasets.ImageFolder(
        traindir,
        transforms.Compose([
            transforms.Resize(size=(IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor(),
            normalize
        ])
    )
    val_dataset = datasets.ImageFolder(
        valdir,
        transforms.Compose([
            transforms.Resize(size=(IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor(),
            normalize
        ])
    )
    test_dataset = datasets.ImageFolder(
        testdir,
        transforms.Compose([
            transforms.Resize(size=(IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor(),
            normalize
        ])
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=workers,
        pin_memory=pin_memory,
        sampler=None
    )
    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=workers,
        pin_memory=pin_memory
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=workers,
        pin_memory=pin_memory
    )
    return train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset

In [ ]:
# Data loading
train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset = data_loader(data_dir, BATCH_SIZE, NUM_WORKERS, False)
print("Training Set: " + str(len(train_loader.dataset)))
print("Validation Set: " + str(len(val_loader.dataset)))
print("Testing Set: " + str(len(test_loader.dataset)))

The next code block is the function that creates our algorithm architecture. 

In [ ]:
# Freeze parameters so we don't backprop through them
hidden_layers = [10240, 1024]
def make_model(structure, hidden_layers, lr, preTrained):
    if structure=="densenet161":
        model = models.densenet161(pretrained=preTrained)
        input_size = 2208
    else:
        model = models.vgg16(pretrained=preTrained)
        input_size = 25088
    output_size = 102
    for param in model.parameters():
        param.requires_grad = False

    classifier = nn.Sequential(OrderedDict([
                              ('dropout',nn.Dropout(0.5)),
                              ('fc1', nn.Linear(input_size, hidden_layers[0])),
                              ('relu1', nn.ReLU()),
                              ('fc2', nn.Linear(hidden_layers[0], hidden_layers[1])),
                              ('relu2', nn.ReLU()),
                              ('fc3', nn.Linear(hidden_layers[1], output_size)),
                              ('output', nn.LogSoftmax(dim=1))
                              ]))

    model.classifier = classifier
    return model

In [ ]:
Pretrainedmodel = make_model(ARCH, hidden_layers, LEARNING_RATE, True)

In [ ]:
# define loss and optimizer
criterion = nn.NLLLoss()
PretrainedmodelOptimizer = optim.Adam(Pretrainedmodel.classifier.parameters(), lr=LEARNING_RATE)

Now we have everything setup and ready to start traning

**In the next section, we will load our trained model to make our results reproducable. You can change the loading path to use your own instead**

In [ ]:
###########################
### DOWNLOAD THE MODELS ###
###########################

## FEEL FREE TO CHANGE THESE PARAMETERS
pretrained_Model_URL = "http://faridnakhle.com/pv/models/RSGAI_DenseNet.zip"
save_data_to = "/content/models/"
pretrained_file_name = "densenet.zip"
#######################################

if not os.path.exists(save_data_to):
    os.makedirs(save_data_to)

print("Downloading models...")  


r = requests.get(pretrained_Model_URL, stream = True, headers={"User-Agent": "Ready, Steady, Go AI"})
with open(save_data_to + pretrained_file_name, "wb") as file: 
    for block in r.iter_content(chunk_size = 1024):
         if block: 
             file.write(block)

## Extract downloaded zip dataset file
print("Model downloaded")  
print("Extracting files...")


with zipfile.ZipFile(save_data_to + pretrained_file_name, 'r') as zip_dataset:
    zip_dataset.extractall(save_data_to)


print("All done!")  


In [ ]:
PRETRAINED_PATH = '/content/models/RSGAI_DenseNet.pth'
def loading_checkpoint(path, pretrained):
    # Loading the parameters
    state = torch.load(path)
    LEARNING_RATE = state['learning_rate']
    structure = state['structure']
    hidden_layers = state['hidden_layers']
    epochs = state['epochs']
    
    # Building the model from checkpoints
    model = make_model(structure, hidden_layers, LEARNING_RATE, pretrained)
    model.class_to_idx = state['class_to_idx']
    model.load_state_dict(state['state_dict'])
    model.eval()
    return model


Pretrainedmodel = loading_checkpoint(PRETRAINED_PATH, True)

# 4. Generating Explanations with LIME

In [ ]:
!pip install mahotas
import itertools
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import mahotas
import cv2
import h5py
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from PIL import Image

## LIME 
from lime import lime_image
from skimage import io
from skimage import img_as_ubyte
from skimage.segmentation import mark_boundaries

In [ ]:
image2explain = '/content/dataset/tomato-dataset/test/Tomato___Late_blight/image (1076)_cropped_1.JPG'

Explanations with Pretrained CNN

In [ ]:
def get_PCNN_image(path):
  image = cv2.imread(path)
  image = cv2.resize(image, (226,226))
  return image

PCNNimg = get_PCNN_image(image2explain)
PCNNimg = cv2.cvtColor(PCNNimg, cv2.COLOR_BGR2RGB)

def get_preprocess_transform():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])     
    transf = transforms.Compose([
        transforms.ToTensor(),
        normalize
    ])    

    return transf    

preprocess_transform = get_preprocess_transform()


def batch_predictPDCNN(images):
    Pretrainedmodel.eval()
    batch = torch.stack(tuple(preprocess_transform(i) for i in images), dim=0)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    Pretrainedmodel.to(device)
    batch = batch.to(device)
    logits = Pretrainedmodel(batch)
    probs = F.softmax(logits, dim=1)
    return probs.detach().cpu().numpy()


explainerPCNN = lime_image.LimeImageExplainer()
explanationPCNN = explainerPCNN.explain_instance(PCNNimg, 
                                         batch_predictPDCNN, # classification function
                                        # top_labels=5, 
                                         hide_color=0, 
                                         num_samples=5000)

In [ ]:
tempPCNN, maskPCNN = explanationPCNN.get_image_and_mask(explanationPCNN.top_labels[0], positive_only=True, num_features=1, hide_rest=False)

In [ ]:
###############################
##     SHOW  EXPLANATION     ##
###############################
tempCNNP, maskCNNP = explanationPCNN.get_image_and_mask(explanationPCNN.top_labels[0], positive_only=False, num_features=1, hide_rest=False)
fig, (ax1) = plt.subplots(1, 1, figsize=(5,5))
ax1.bbox_inches='tight'
ax1.pad_inches = 0
ax1.axis('off')
plt.subplots_adjust(wspace=0, hspace=0)
plt.imshow(mark_boundaries(tempCNNP, maskCNNP))